<a href="https://colab.research.google.com/github/gemma24/ISYS5002-assessment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Client's requirements
This notebook able to:
* Prompts for a company's stock market ticker symbol
* Scrapes the Yahoo finance website to extract info on the key company
* Calculate the each executive's expected income tax based on the ATO tax table
* Data can be saved as a CSV or SQLite database
* Follow PEP8 conventions



# Problem Solving Methodology
In the process of problem-solving, there are five key steps to follow:
1. State the problem clearly
2. Describe the input and output information
3. Work the problem by hand 
4. Develop an algorithm
5. Test solution with a variety of data.
# Step 1: State the problem 
* Find out what is ticker symbol
* Find the Yahoo finance wbesite and how to scrape?
* How to access each executive's information?
* How to calculate the each executive's expected income tax?
* How to save a data as a CSV and SQlite database








# Step 2: Describe the input and output information
## Website 

[Yahoo Finance](https://au.finance.yahoo.com/)

## Inputs

* Prompt for a ticker symbol

## Processing

* Get income from webpage
* Calculate Tax (using income)

## Outputs

* Save to CSV file 
* Save to SQLIte 




### Exploring the Website

Stock Ticker Symbol:  A stock symbol is a unique series of letters assigned to a security for trading purposes.  From https://www.investopedia.com/terms/s/stocksymbol.asp

Ticker can be for any company on the exchange

Note: once found page, need to click on the 'Profile' tab.

For TGT the profile page is:

https://au.finance.yahoo.com/quote/TGT/profile?p=TGT

For WOW.AX:

https://au.finance.yahoo.com/quote/WOW.AX/profile?p=WOW.AX


For COL.AX (Coles Group)

https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX
